# Todo:

* Remodel the agent to ~PPO~ (looks like PPO isn't implemented in keras-rl) DDQN leveraging its own LSTM, test against DQN. Might be here tho: https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/rl/ipynb/ppo_cartpole.ipynb
 
* If that isn't successful, we can try including those values with the LSTM predictions in the data that will go the Agent
* Can an SDAE feature extraction make for a better lstm prediction?
* Consider that our applicaton of Attention in the LSTMs is wrong :( fix it.
* Find an article discussing LSTMs as feature extractors
* Good read: https://arxiv.org/pdf/2212.02721.pdf

More sources:

* LSTM with CONV1 https://shivapriya-katta.medium.com/time-series-forecasting-using-conv1d-lstm-multiple-timesteps-into-future-acc684dcaaa

* LSTMs and Attention: https://medium.com/swlh/a-simple-overview-of-rnn-lstm-and-attention-mechanism-9e844763d07b




In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')

Mounted at /content/drive


In [ ]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

In [ ]:
!pip install keras-rl2

In [ ]:
!pip install -Uqq ipdb
import ipdb


In [ ]:
%pdb off

# Data

In [ ]:
# Executes this notebook in our space, making all of its functions/globals availablev
_models = None
replace = False

if 'models' in globals():
  _models = models
  should_load = models_loaded
  replace = True
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'

if replace:
  models_loaded = should_load
  models = _models

In [ ]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

In [ ]:
def download_stocks():
  # We don't want to train against crypto
  return get_all_stock_timerseries_for_csv("training_tickers3.csv", 3500,-1)

In [ ]:
LENGTH_OF_STOCK_TRAINGING_DATA = 145 #I might need to fix this, but the model is tied to the number of symbols we trained on
def download_crypto():
  return get_all_product_timeseries(-1, 180, LENGTH_OF_STOCK_TRAINGING_DATA)

In [ ]:
def save_dict(product_data, crypto=False):
  # Write each dataframe to disk
  for key, df in product_data.items():
    df.to_csv(data_path + "/dqn-preds-" + key + ".csv", index=False)

  # Convert the keys of the dictionary to a dataframe
  keys_df = pd.DataFrame({"keys": list(product_data.keys())})

  # Save the keys dataframe to disk as a CSV file
  if crypto:
    keys_df.to_csv(data_path + "/dqn-preds-keys-crypto.csv", index=False)
  else:
    keys_df.to_csv(data_path + "/dqn-preds-keys.csv", index=False)

def load_dict(crypto=False):
  if crypto:
    keys = pd.read_csv(data_path + "/dqn-preds-keys-crypto.csv")
  else:
    keys = pd.read_csv(data_path + "/dqn-preds-keys.csv")
  product_data = {}
  # Iterate over the keys
  for index, row in keys.iterrows():
    key = row["keys"]
    product_data[key] = pd.read_csv(data_path + "/dqn-preds-" + key + ".csv")
  return product_data

In [ ]:
def transform_to_predictions(product_data):
  product_data_= append_predictions(product_data, True)
  return product_data_

In [ ]:
# we need to generate a massive action set actions is an array of ints, so we need some sort of mapping.
actions = []
action_map = []
stock_actions = [0,1,-1] #hold, buy, short
product_map = {}
index = 0
product_id = 0

from_disk = True
use_crypto = True

# this is dumb, but since I trained and saved the model with 145 different 
# stocks to select from, I now have to use only that many products
MAGIC_SHAPE_NUMBER = 145

if from_disk:
  product_data = load_dict(use_crypto)
  first_key = list(product_data.keys())[0]
  length = len(product_data[first_key])
elif use_crypto:
  print("load models? ", models_loaded)
  if not models_loaded:
    load_all_models()
  print("models loaded? ", models_loaded)
  product_data = download_crypto()
  product_data = transform_to_predictions(product_data)
  save_dict(product_data, True)
else:
  if not models_loaded:
    load_all_models()
  product_data = download_stocks()
  product_data = transform_to_predictions(product_data)
  save_dict(product_data)

min_len = min(len(df) for df in product_data.values())
print("symbols:", len(product_data.keys()))
print("min length:", min_len)
for name, df in product_data.items():
    product_data[name] = df.head(min_len)



symbols: 145
min length: 180


In [ ]:
print(len(product_data.keys()), " ", MAGIC_SHAPE_NUMBER)
assert(len(product_data.keys()) == MAGIC_SHAPE_NUMBER)

145   145


# DQN Agent


In [ ]:
for product, data in product_data.items():
  product_map[product] = product_id
  product_id+=1
  for action in stock_actions:
    actions.append(index) #action for the AI is number: index
    action_map.append([product, action]) #our lookup for the env will be product + buy, sell, hold
    index+=1

In [ ]:
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [ ]:
class BackTestingEnv():
    def __init__(self, product_data):
        # Initialize the environment and retrieve stock data from a data source
        self.product_data = product_data
        self.current_index = 0
        self.cash = 5000
        self.position_shares = 0
        self.position_value = 0
        first_key = list(product_data.keys())[0]
        self.final = len(product_data[first_key])
        self.current_product = None
        self.ledger = self.make_ledger_row()
        self.slippage = .01
        self.fee = .0025

    def make_ledger_row(self):
      ledger = pd.DataFrame()
      ledger["Date"] = []
      ledger["Product"] = []
      ledger["Side"] = []
      ledger["Action"]  = []
      ledger["Price"] = []
      ledger["Fee"] = []
      return ledger

    def reset(self):
        # Reset the environment and return the first observation
        info("Reset!!!!")
        self.current_index = 0
        self.cash = 5000
        self.position_shares = 0
        self.position_value = 0
        self.current_product = None
        self.ledger = self.make_ledger_row()
        return self.env_block()

    def step(self, action):
        # Advance the environment by one time step and return the observation, reward, and done flag
        self.current_index += 1
        info("step:", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)

        if (self.current_index >= self.final - 1 or self.calc_reward() <= 0):
          info("********MARKING DONE", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)
          if (self.current_product != None):
            info("done so closing: ", self.current_product)
            self.close_position()
          reward = self.calc_reward()
          return self.env_block(), reward, True, {}
        else:
          done = False

        # convert action to product and action
        [product, stock_action] = action_map[action]

        if ((product == self.current_product and stock_action == 1) or stock_action == 0):
          info("holding: ", self.current_product)
          self.update_position_value()
          reward = self.calc_reward()
          return self.env_block(), reward, done, {}
        elif (product == self.current_product and stock_action == -1):
          info("closing: ", self.current_product)
          self.close_position()
          reward = self.calc_reward()
          return self.env_block(), reward, done, {}
        elif (product != self.current_product and self.current_product != None and stock_action == 1):
          info("closing and opening: ", self.current_product, product)
          self.close_position()
          self.open_position(product)
          reward = self.calc_reward()
          return self.env_block(), reward, done, {}
        elif (self.current_product == None and stock_action == 1):
          info("opening: ", self.current_product, product)
          self.open_position(product)
          reward = self.calc_reward()
          return self.env_block(), reward, done, {}
        else:
          info("invalid")
          reward = self.calc_reward()
          return self.env_block(), reward, done, {}

    def calc_reward(self):
      info("current reward: ", self.position_value + self.cash)
      return self.position_value + self.cash

    def get_price_with_slippage(self, price):
        return price + (price * self.slippage)

    def open_position(self, product):
      info("opening position: ", product)
      df = self.product_data[product]
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      self.position_shares = math.floor(self.cash/price)
      self.position_value  = self.position_shares * price
      self.cash = 0
      self.current_product = product
      ledger_row = self.make_ledger_row()
      ledger_row["Product"] = [product]
      ledger_row["Side"] = [1]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.fee]
      self.ledger = self.ledger.append(ledger_row)


    def close_position(self):
      info("closing position: ", product)
      df = self.product_data[self.current_product]
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.position_shares
      self.position_shares = 0
      self.position_value = 0
      self.cash = value
      ledger_row = self.make_ledger_row()
      ledger_row["Product"] = [self.current_product]
      ledger_row["Side"] = [1]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.cash * self.fee]
      self.ledger = self.ledger.append(ledger_row)
      self.current_product = None

    def update_position_value(self):
      if self.current_product == None:
        return
      df = self.product_data[self.current_product]
      row = df.iloc[self.current_index,:]
      self.position_value = row["Close"] * self.position_shares
    

    def render(self, mode='human'):
        # Render the environment for human consumption
        pass

    def env_block(self):
      selected_rows_list = []
      for product, df in self.product_data.items():
        row = df.iloc[self.current_index,:]
        row["product"] = product_map[product]
        row = row.drop("Close")
        selected_rows_list.append(row.values)
      np_selected_rows = np.array(selected_rows_list)
      return np_selected_rows

env = BackTestingEnv(product_data)
first_key = list(product_data.keys())[0]
# NOTE: We drop Close (in favor of scaled_close) but add product so this is a wash
cols = len(product_data[first_key].columns)
symbols = len(env.product_data)


In [ ]:
import math
#np.random.seed(123)


# study: https://github.com/tensorneko/keras-rl2/blob/master/examples/dqn_cartpole.py



nb_actions = len(actions) # Number of possible actions
print("ruh oh: ", symbols)
weights_filename = model_path + '/dqn_trader_weights.h5f'
midpoint_filename = model_path + "/dqn_trader_weights_250000.h5f"

    
# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(1, symbols,cols)))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(nb_actions))
model.add(keras.layers.Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=1000000, window_length=1)


# Select a policy. We use eps-greedy action selection, which means that a random action is selected
# with probability eps. We anneal eps from 1.0 to 0.1 over the course of 1M steps. This is done so that
# the agent initially explores the environment (high eps) and then gradually sticks to what it knows
# (low eps). We also set a dedicated eps value that is used during testing. Note that we set it to 0.05
# so that the agent still performs some random actions. This ensures that the agent cannot get stuck.
#policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
#                              nb_steps=1000000)
policy = BoltzmannQPolicy()

# The trade-off between exploration and exploitation is difficult and an on-going research topic.
# If you want, you can experiment with the parameters or use a different policy. Another popular one
# is Boltzmann-style exploration:
# policy = BoltzmannQPolicy(tau=1.)
# Feel free to give it a try!

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
              target_model_update=1e-2, policy=policy)

dqn.compile(Adam(lr=1e-3), metrics=['mae'])
checkpoint_weights_filename = model_path + '/dqn_trader_weights_{step}.h5f'
log_filename = data_path + '/dqn_{args.env_name}_log.json'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
train = False
if train:
  if os.path.exists(midpoint_filename):
    dqn.model.load_weights(midpoint_filename)
  
  dqn.fit(env, callbacks=callbacks, nb_steps=1750000, visualize=False,  log_interval=10000, verbose=2)

  # After training is done, we save the final weights one more time.
  dqn.save_weights(weights_filename, overwrite=True)

  
else:
    dqn.load_weights(weights_filename)
    dqn.test(env, nb_episodes=10, visualize=False)

NameError: ignored

In [ ]:
env.calc_reward()


current reward:  5336.043211


5336.043211